In [127]:
import numpy as np
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister, transpile
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

import warnings
warnings.filterwarnings('ignore')

In [128]:
def create_oracle_marked(marked_indices, num_qubits):
    oracle = QuantumCircuit(num_qubits)
    for i in marked_indices:
        bits = f"{i:0{num_qubits}b}"
        indices_to_flip = [j for j, b in enumerate(bits[::-1]) if b == '0']
        if indices_to_flip:
            oracle.x(indices_to_flip)
        oracle.mcx(list(range(num_qubits - 1)), num_qubits - 1)
        if indices_to_flip:
            oracle.x(indices_to_flip)
    return oracle

def durr_hoyer(exp_array):
    num_items = len(exp_array)
    num_qubits = int(np.ceil(np.log2(num_items)))
    min_value = float(0)
    min_index = None
    final_counts = None
    backend = AerSimulator()
    for threshold in sorted(set(exp_array)):
        marked = [i for i,v in enumerate(exp_array) if v <= threshold]
        if not marked:
            continue

        index_register = QuantumRegister(num_qubits)
        cls_register = ClassicalRegister(num_qubits)
        qc = QuantumCircuit(index_register, cls_register)
        qc.h(index_register)
        oracle = create_oracle_marked(marked, num_qubits)
        
        qc.compose(oracle, index_register, inplace=True)
        qc.h(index_register)
        qc.x(index_register)
        qc.h(index_register[-1])
        qc.mcx(list(range(num_qubits-1)), index_register[-1])
        qc.h(index_register[-1])
        qc.x(index_register)
        qc.h(index_register)
        
        qc.measure(index_register, cls_register)
        result = backend.run(transpile(qc, backend=backend)).result()
        counts = result.get_counts()
        if counts:
            most_frequent_index = max(counts, key=counts.get)
            current_index = int(most_frequent_index, 2)
            current_value = exp_array[current_index]
            if current_value > min_value:
                min_value = current_value
                min_index = current_index
                final_counts = counts
    if min_index is None:
        print("No better results found.")
        return None
    return qc, min_value, min_index, final_counts

In [129]:
arr = np.random.normal(loc=5, scale=2, size=64) 
# num_outliers = 10
# outlier_indices = np.random.choice(64, num_outliers, replace=False)
# arr[outlier_indices] = np.random.uniform(50, 100, num_outliers)

In [130]:
arr

array([ 6.14032211,  4.4189045 ,  6.41628139,  7.01665427,  6.39400528,
        2.88990199,  6.31678062,  4.14970264,  1.75775781,  3.69673835,
        6.43184667,  7.23801583,  5.66124445,  2.92156823,  4.23863512,
        6.38466783,  4.38372584,  4.35599448,  6.12326707,  6.28830705,
        2.53074145,  3.58568296,  8.19028594,  3.42411768, 10.55863205,
        4.92569377,  6.22295861,  3.70360201,  4.57569674,  6.13658071,
        7.93936   ,  5.23635239,  5.92078169,  4.16964428,  7.57324807,
        2.14428823,  3.83722244,  7.88136612,  4.64118175,  3.8383124 ,
        5.1779188 ,  6.2581076 ,  2.70931524,  4.41126526,  5.85738117,
        6.7811464 ,  8.12604237,  3.12144736,  2.95190182,  4.45307134,
        3.82560283,  2.63394637,  0.81688136,  2.96331127,  4.67853769,
        4.70391249,  3.64357032,  6.42143083,  5.75315488,  5.93259642,
        4.92262511,  5.82409699,  9.78207615,  6.03488311])

In [131]:
len(arr)

64

In [132]:
exp_array = arr
l=[]
for i in range(int(np.ceil(np.sqrt(len(exp_array))))):
    qc, min_value, min_index, final_counts = durr_hoyer(exp_array)
    print(f"Grover approximate maximum: {min_value}, at index {min_index}")
    l.append(min_value)

Grover approximate maximum: 8.190285935370357, at index 22
Grover approximate maximum: 10.558632051368402, at index 24
Grover approximate maximum: 8.190285935370357, at index 22
Grover approximate maximum: 10.558632051368402, at index 24
Grover approximate maximum: 10.558632051368402, at index 24
Grover approximate maximum: 10.558632051368402, at index 24
Grover approximate maximum: 10.558632051368402, at index 24
Grover approximate maximum: 9.782076151548313, at index 62


In [133]:
max(arr)

np.float64(10.558632051368402)